In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
url = 'http://www.consob.it/mainen/intermediares/financial_salesman/historical_register.htm'
html = requests.get(url).text

In [3]:
bs = BeautifulSoup(html,'lxml')

In [4]:
articles = []
for href in bs.findAll(href=re.compile(r'xsl$')):
    articles.append(href.get('href'))

In [5]:
root = 'http://www.consob.it'

In [102]:
testurl = root+articles[1]
testhtml = requests.get(testurl).text

In [100]:
# get html table mauraly, worth 500$ 
def get_table(html):
    testbs = BeautifulSoup(html,'lxml')
    table = testbs.find('table')
    rows=list()
    for row in table.findAll("tr"):
       rows.append(row)
    result = list()
    i=0
    for line in rows:
        td = line.findAll('td')
        if re.findall('^[A-Z]',td[0].text):
            result.append((i/6,td[0].text,td[1].text.replace('\n','')))
            i+=1
    df = pd.DataFrame(result)
    df_p = df.pivot(index=0, columns=1, values=2)
    return df_p.reset_index(drop = True)

In [110]:
df_fi = pd.DataFrame()
for uri in articles[:5]:
    print uri,'...',
    url = root+uri
    html = requests.get(url).text
    df = get_table(html)
    df_fi = pd.concat([df_fi,df],ignore_index=True)
    print 'ok'

/documenti/promotori/albo/abruzzo.xml?xsl=albopromotorien.xsl ... ok
/documenti/promotori/albo/basilicata.xml?xsl=albopromotorien.xsl ... ok
/documenti/promotori/albo/calabria.xml?xsl=albopromotorien.xsl ... ok
/documenti/promotori/albo/campania.xml?xsl=albopromotorien.xsl ... ok
/documenti/promotori/albo/emilia.xml?xsl=albopromotorien.xsl ... ok
